In [2]:
import psycopg2
import pandas as pd

In [40]:
# Connect to the transactional PostgreSQL database
conn_transactional = psycopg2.connect(
    host="localhost",
    port="5433",
    database="transactional_db",
    user="user",
    password="password"
)

# Connect to the data warehouse PostgreSQL database
conn_dwh = psycopg2.connect(
    host="localhost",
    port="5434",
    database="dwh_db",
    user="user",
    password="password"
)


In [41]:
cur_transactional = conn_transactional.cursor()
cur_dwh = conn_dwh.cursor()

# Print a success message as test
print("Connected to both transactional and DWH databases successfully!")

Connected to both transactional and DWH databases successfully!


In [33]:
# SQL queries to create tables
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
    user_id SERIAL PRIMARY KEY,
    user_name VARCHAR(50),
    email VARCHAR(50) UNIQUE,
    wallet_balance NUMERIC(10, 2),
    phone_number VARCHAR(20),
    address VARCHAR(100)
);
"""
create_author_table = """
CREATE TABLE IF NOT EXISTS authors (
    author_id SERIAL PRIMARY KEY,
    author_name VARCHAR(50),
    email VARCHAR(50) UNIQUE,
    nationality VARCHAR(50)
);
"""
create_book_table = """
CREATE TABLE IF NOT EXISTS books (
    book_id SERIAL PRIMARY KEY,
    title VARCHAR(100),
    publish_date DATE,
    isbn VARCHAR(20),
    genre VARCHAR(50),
    price NUMERIC(10, 2)
);
"""
create_book_author_table = """
CREATE TABLE IF NOT EXISTS book_author (
    book_id INT,
    author_id INT,
    CONSTRAINT fk_book_id FOREIGN KEY (book_id) REFERENCES books(book_id),
    CONSTRAINT fk_author_id FOREIGN KEY (author_id) REFERENCES authors(author_id),
    PRIMARY KEY (book_id, author_id)
);
"""
create_review_table = """
CREATE TABLE IF NOT EXISTS reviews (
    review_id SERIAL PRIMARY KEY,
    user_id INT,
    book_id INT,
    rate INT CHECK (rate >= 1 AND rate <= 5),
    review_text TEXT,
    review_date DATE,
    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (book_id) REFERENCES books(book_id)
);
"""
create_order_table = """
CREATE TABLE IF NOT EXISTS orders (
    order_id SERIAL PRIMARY KEY,
    user_id INT,
    order_date DATE,
    total_amount NUMERIC(10, 2),
    order_created DATE,
    order_completed DATE,
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);
"""
create_order_book_table = """
CREATE TABLE IF NOT EXISTS order_book (
    order_id INT,
    book_id INT,
    quantity INT,
    PRIMARY KEY (order_id, book_id),
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (book_id) REFERENCES books(book_id)
);
"""

In [34]:
# Execute the table creation queries on the transacional database
cur_transactional.execute(create_users_table)
cur_transactional.execute(create_book_table)
cur_transactional.execute(create_author_table)
cur_transactional.execute(create_book_author_table)
cur_transactional.execute(create_review_table)
cur_transactional.execute(create_order_table)
cur_transactional.execute(create_order_book_table)

In [35]:
conn_transactional.commit()

In [36]:
try:
    # SQL statements to create the star schema tables
    create_dim_user = """
    CREATE TABLE IF NOT EXISTS dim_user (
        user_id SERIAL PRIMARY KEY,
        user_name VARCHAR(100),
        email VARCHAR(100) UNIQUE,
        phone_number VARCHAR(20),
        address VARCHAR(255)
    );
    """

    create_dim_author = """
    CREATE TABLE IF NOT EXISTS dim_author (
        author_id SERIAL PRIMARY KEY,
        author_name VARCHAR(100),
        nationality VARCHAR(50)
    );
    """

    create_dim_book = """
    CREATE TABLE IF NOT EXISTS dim_book (
        book_id SERIAL PRIMARY KEY,
        title VARCHAR(255),
        author_id INT,
        genre VARCHAR(50),
        price NUMERIC(10, 2),
        publish_date DATE,
        FOREIGN KEY (author_id) REFERENCES dim_author(author_id)
    );
    """

    create_dim_time = """
    CREATE TABLE IF NOT EXISTS dim_time (
        time_id SERIAL PRIMARY KEY,
        date DATE,
        year INT,
        month INT,
        day INT,
        quarter INT
    );
    """

    create_fact_sales = """
    CREATE TABLE IF NOT EXISTS fact_sales (
        sales_id SERIAL PRIMARY KEY,
        book_id INT,
        user_id INT,
        time_id INT,
        quantity_sold INT,
        total_amount NUMERIC(10, 2),
        FOREIGN KEY (book_id) REFERENCES dim_book(book_id),
        FOREIGN KEY (user_id) REFERENCES dim_user(user_id),
        FOREIGN KEY (time_id) REFERENCES dim_time(time_id)
    );
    """

    # Execute the table creation queries for the star schema
    cur_dwh.execute(create_dim_user)
    cur_dwh.execute(create_dim_author)
    cur_dwh.execute(create_dim_book)
    cur_dwh.execute(create_dim_time)
    cur_dwh.execute(create_fact_sales)

    # Commit the changes
    conn_dwh.commit()
    print("Star schema tables created successfully in the DWH database!")

except Exception as e:
    print(f"Error occurred: {e}")
    conn_dwh.rollback()

Star schema tables created successfully in the DWH database!


In [37]:
#alter and edit some columns to fit the data
cur_transactional.execute("ALTER TABLE authors ALTER COLUMN author_name TYPE VARCHAR(100);")
cur_transactional.execute("ALTER TABLE users ALTER COLUMN user_name TYPE VARCHAR(100);")
cur_transactional.execute("ALTER TABLE authors ALTER COLUMN email TYPE VARCHAR(100);")
cur_transactional.execute("ALTER TABLE books ALTER COLUMN genre TYPE VARCHAR(100);")


In [39]:
conn_transactional.close()  
conn_dwh.close()